<a href="https://colab.research.google.com/github/kwangwonland/kfq_ML_2023/blob/main/0725_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup
import os,pathlib, shutil,random



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  6899k      0  0:00:11  0:00:11 --:--:-- 15.9M


In [2]:
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir/"val"
train_dir = base_dir/"train"
for category in ("neg","pos"):
  os.makedirs(val_dir/category)
  files = os.listdir(train_dir/category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2*len(files))
  val_files = files[:num_val_samples]
  for fname in val_files:
    shutil.move(train_dir/category/fname,
                val_dir/category/fname)

In [3]:
from tensorflow import keras

train_ds = keras.utils.text_dataset_from_directory('aclImdb/train',
                                                   batch_size=32)
val_ds = keras.utils.text_dataset_from_directory('aclImdb/val',
                                                   batch_size=32)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test',
                                                   batch_size=32)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [4]:
from keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [5]:
max_length=600
max_tokens=20000

tv = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_only_train_ds = train_ds.map(lambda x,y: x)

tv.adapt(text_only_train_ds) #vacabulary 형성

In [6]:
int_train_ds = train_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
int_val_ds = val_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)
int_test_ds = test_ds.map(
    lambda x,y: (tv(x),y),
    num_parallel_calls=4
)

In [7]:
import tensorflow as tf

inputs = keras.Input(shape=(None,),dtype='int64')
embedded = tf.one_hot(inputs,depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model = keras.Model(inputs,outputs)
model.compile(loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [ ]:
mcp = keras.callbacks.ModelCheckpoint('one_hot_bidir_lstm.keras',
                                      save_best_only=True)
model.fit(int_train_ds.cache(),
          validation_data=int_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('one_hot_bidir_lstm.keras')
model.evaluate(int_test_ds)


Epoch 1/10
 17/625 [..............................] - ETA: 3:14:52 - loss: 0.6935 - accuracy: 0.4835

In [ ]:
mcp = keras.callbacks.ModelCheckpoint('embedding_bidir_lstm.keras',
                                      save_best_only=True)
model.fit(int_train_ds.cache(),
          validation_data=int_val_ds.cache(),
          epochs=10,
          callbacks=[mcp])
model = keras.models.load_model('embedding_bidir_lstm.keras')
model.evaluate(int_test_ds)

Epoch 1/10
286/625 [============>.................] - ETA: 1:30:11 - loss: 0.6644 - accuracy: 0.5925

In [15]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/acllmdb_V1.tar.gz

--2023-07-25 07:21:13--  https://ai.stanford.edu/~amaas/data/sentiment/acllmdb_V1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-07-25 07:21:14 ERROR 404: Not Found.



In [8]:
!tar -xf acllmdb_v1.tar.gz

tar: acllmdb_v1.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [9]:
import tensorflow as tf
from tensorflow import keras

In [12]:
dataset=keras.utils.text_dataset_from_directory(
    directory='acllmdb',
    label_mode=None,
    batch_size=256
)
dataset=dataset.map(lambda x:tf.strings.regex_replace(x,'<br /',' '))

Found 0 files belonging to 1 classes.


ValueError: ignored